In [1]:
import requests
import pandas as pd

def get_product_detail(pid):
    url = 'https://api.bunjang.co.kr/api/pms/v1/products-detail/{}?viewerUid=-1'.format(pid)
    response = requests.get(url) #url 요청 보내고 응답 받아옴
    data = response.json().get('data', {}).get('product', {})
    return {
            'pid': data.get('pid', ''),
            'name': data.get('name', ''),
            'description': data.get('description', ''),
            'price': data.get('price', ''),
            'qty': data.get('qty', ''),
            'saleStatus': data.get('saleStatus', ''),
            'status': data.get('status', ''),
            'imageUrl': data.get('imageUrl', ''),
            'keywords' : data.get('keywords',''),
            'address' : data.get('geo', {}).get('address', ''),
            'label' : data.get('geo', {}).get('label', '')
        }

def bunjang(key_word, pages, items_number):
    """
    items_number 한 페이지당 가지고 오는 갯수인데 기본값은 96이였습니다!
    """
    data = []
    for page in range(pages):
        url = 'https://api.bunjang.co.kr/api/1/find_v2.json?order=date&n={}&page={}&req_ref=search&q={}&stat_device=w&stat_category_required=1&version=4'.format(items_number,page,key_word)
        response = requests.get(url)
        datas = response.json()['list']
        for piddata in datas:
            pid = piddata['pid']
            detail_data = get_product_detail(pid)
            if detail_data is not None:
                data.append(detail_data)
    
    df = pd.DataFrame(data)

    df['link'] = 'https://m.bunjang.co.kr/products/' + df['pid'].astype(str)
    df['market'] = '번개장터'
    df['keyword'] = key_word

    df.to_csv('bunjang_data.csv', index=False, encoding='utf-8-sig') # encoding='utf-8'로 하니깐 계속 파일이 깨져가지고 'utf-8(bom)'으로 지정해놓은것입니다!!



In [2]:
bunjang('노트북',1,30)